<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc" style="margin-top: 1em;"><ul class="toc-item"><li><span><a href="#Prepare" data-toc-modified-id="Prepare-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Prepare</a></span><ul class="toc-item"><li><span><a href="#Import-Library" data-toc-modified-id="Import-Library-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Import Library</a></span></li><li><span><a href="#Fixed-Point" data-toc-modified-id="Fixed-Point-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>Fixed Point</a></span></li><li><span><a href="#Newton's-Method" data-toc-modified-id="Newton's-Method-1.3"><span class="toc-item-num">1.3&nbsp;&nbsp;</span>Newton's Method</a></span></li><li><span><a href="#Secant-Method" data-toc-modified-id="Secant-Method-1.4"><span class="toc-item-num">1.4&nbsp;&nbsp;</span>Secant Method</a></span></li><li><span><a href="#Quasi-Newton-Method" data-toc-modified-id="Quasi-Newton-Method-1.5"><span class="toc-item-num">1.5&nbsp;&nbsp;</span>Quasi-Newton Method</a></span></li><li><span><a href="#Steepest-Descent" data-toc-modified-id="Steepest-Descent-1.6"><span class="toc-item-num">1.6&nbsp;&nbsp;</span>Steepest Descent</a></span></li></ul></li><li><span><a href="#-Run-" data-toc-modified-id="-Run--2"><span class="toc-item-num">2&nbsp;&nbsp;</span><font color="orange"> Run <font></font></font></a></span><ul class="toc-item"><li><span><a href="#1-Dim" data-toc-modified-id="1-Dim-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>1-Dim</a></span><ul class="toc-item"><li><span><a href="#Fixed-Point" data-toc-modified-id="Fixed-Point-2.1.1"><span class="toc-item-num">2.1.1&nbsp;&nbsp;</span>Fixed Point</a></span></li><li><span><a href="#Newton's-method" data-toc-modified-id="Newton's-method-2.1.2"><span class="toc-item-num">2.1.2&nbsp;&nbsp;</span>Newton's method</a></span></li><li><span><a href="#Secant-Method" data-toc-modified-id="Secant-Method-2.1.3"><span class="toc-item-num">2.1.3&nbsp;&nbsp;</span>Secant Method</a></span></li></ul></li><li><span><a href="#Multi-Dim" data-toc-modified-id="Multi-Dim-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>Multi-Dim</a></span><ul class="toc-item"><li><span><a href="#Fixed-Point" data-toc-modified-id="Fixed-Point-2.2.1"><span class="toc-item-num">2.2.1&nbsp;&nbsp;</span>Fixed Point</a></span></li><li><span><a href="#Newton's-method" data-toc-modified-id="Newton's-method-2.2.2"><span class="toc-item-num">2.2.2&nbsp;&nbsp;</span>Newton's method</a></span></li><li><span><a href="#Quasi-Newton-method" data-toc-modified-id="Quasi-Newton-method-2.2.3"><span class="toc-item-num">2.2.3&nbsp;&nbsp;</span>Quasi-Newton method</a></span></li><li><span><a href="#Steepest-Descent" data-toc-modified-id="Steepest-Descent-2.2.4"><span class="toc-item-num">2.2.4&nbsp;&nbsp;</span>Steepest Descent</a></span></li></ul></li></ul></li></ul></div>

You can directly go to **Chapter 2 (Run)** to look experiment results.

# Prepare

## Import Library

In [1]:
import numpy as np
from numpy import linalg
from abc import abstractmethod
import pandas as pd
import math

pd.options.display.float_format = '{:,.8f}'.format
np.set_printoptions(suppress=True, precision=8)

TOR = pow(10.0, -9)
MAX_ITR = 45

## Fixed Point

In [2]:
class FixedPointMethod(object):

    def __init__(self):
        return

    @abstractmethod
    def f(self, x):
        return NotImplementedError('Implement f()!')

    @abstractmethod
    def run(self, x):
        return NotImplementedError('Implement run()!')

In [3]:
class FixedPoint1D(FixedPointMethod):

    def __init__(self):
        super(FixedPointMethod, self).__init__()

    def f(self, x):
        return math.cos(x)

    def g1(self, x):
        return np.array(x - pow(x, 3) - 4 * pow(x, 2) + 10)

    def g2(self, x):
        return np.array(math.sqrt(10 / x - 4 * x))

    def g3(self, x):
        return np.array(math.sqrt(10 - pow(x, 3)) / 2)

    def g4(self, x):
        return np.array(math.sqrt(10 / (4 + x)))

    def g5(self, x):
        return np.array(x - (pow(x, 3) + 4 * pow(x, 2) - 10) / (3 * pow(x, 2) + 8 * x))

    def run2(self, x0):
        df = pd.DataFrame(columns=['(FP) f(x)'])
        row = len(df)
        x = x0
        df.loc[row] = [x]
        for k in range(MAX_ITR):
            try:
                y = self.f(x)
            except ValueError:
                break
            residual = math.fabs(x - y)
            x = y

            row = len(df)
            df.loc[row] = [y]
            if residual < TOR or x > 1e9:
                break
        return df

    def run(self, x0):
        """
        given x_0 in R^3 as a starting point.

        :param x: x_0 as described
        :return: the minimizer x* of f
        """
        g = [self.g1, self.g2, self.g3, self.g4, self.g5]

        total_df = None
        for j in range(len(g)):
            df = pd.DataFrame(columns=['g' + str(j + 1) + '(x)'])
            row = len(df)
            x = x0
            df.loc[row] = [x]
            for k in range(MAX_ITR):
                try:
                    y = np.array(g[j](x))
                except ValueError:
                    break
                residual = math.fabs(x - y)
                x = y

                row = len(df)
                df.loc[row] = [y]
                if residual < TOR or x > 1e9:
                    break
            total_df = df if total_df is None else pd.concat([total_df, df], axis=1)
        return total_df

In [4]:
class FixedPoint(FixedPointMethod):

    def __init__(self):
        super(FixedPointMethod, self).__init__()

    def f(self, x):
        sol = np.zeros(len(x))
        sol[0] = math.cos(x[1] * x[2]) / 3.0 + 1.0 / 6.0
        sol[1] = math.sqrt(x[0] * x[0] + math.sin(x[2]) + 1.06) / 9.0 - 0.1
        sol[2] = -math.exp(-x[0] * x[1]) / 20.0 - (10.0 * math.pi - 3.0) / 60.0
        return sol

    def run(self, x):
        """
        given x_0 in R^3 as a starting point.

        :param x: x_0 as described
        :return: the minimizer x* of f
        """
        df = pd.DataFrame(columns=['x' + str(i + 1) for i in range(len(x))] + ['residual', 'actual-residual'])

        row = len(df)
        df.loc[row] = [xe for xe in x] + [np.nan, np.nan]

        while True:
            y = self.f(x)
            residual = linalg.norm(x - y, np.inf)
            x = y

            row = len(df)
            df.loc[row] = [ye for ye in y] + [residual, np.nan]
            if residual < TOR:
                break

        for i in range(len(df)):
            xk = np.array([df.loc[i][j] for j in range(len(x))])
            df.loc[i][4] = linalg.norm(x - xk, np.inf)

        return df

In [5]:
class FixedPointAcceleration(FixedPointMethod):

    def __init__(self):
        super(FixedPointMethod, self).__init__()

    def f(self, x):
        sol = np.zeros(len(x))
        sol[0] = math.cos(x[1] * x[2]) / 3.0 + 1.0 / 6.0
        sol[1] = math.sqrt(sol[0] * sol[0] + math.sin(x[2]) + 1.06) / 9.0 - 0.1
        sol[2] = -math.exp(-sol[0] * sol[1]) / 20.0 - (10.0 * math.pi - 3.0) / 60.0
        return sol

    def run(self, x):
        """
        given x_0 in R^3 as a starting point.

        :param x: x_0 as described
        :return: the minimizer x* of f
        """
        df = pd.DataFrame(columns=['x' + str(i + 1) for i in range(len(x))] + ['residual', 'actual-residual'])

        row = len(df)
        df.loc[row] = [xe for xe in x] + [np.nan, np.nan]
        while True:
            y = self.f(x)
            residual = linalg.norm(x - y, np.inf)
            x = y

            row = len(df)
            df.loc[row] = [ye for ye in y] + [residual, np.nan]
            if residual < TOR:
                break

        for i in range(len(df)):
            xk = np.array([df.loc[i][j] for j in range(len(x))])
            df.loc[i][4] = linalg.norm(x - xk, np.inf)

        return df

## Newton's Method

In [6]:
class NewtonMethod(object):

    def __init__(self):
        return

    @abstractmethod
    def f(self, x):
        return NotImplementedError('Implement f()!')

    @abstractmethod
    def jacobian(self, x):
        return NotImplementedError('Implement jacobian()!')

    @abstractmethod
    def run(self, x):
        return NotImplementedError('Implement run()!')

In [7]:
class Newton1D(NewtonMethod):

    def __init__(self):
        super(NewtonMethod, self).__init__()

    def f(self, x):
        return math.cos(x) - x

    def jacobian(self, x):
        return -math.sin(x) - 1

    def run(self, x0):
        df = pd.DataFrame(columns=['(NT) f(x)'])
        row = len(df)
        x = x0
        df.loc[row] = [x]
        for k in range(MAX_ITR):
            try:
                y = x - self.f(x) / self.jacobian(x)
            except ValueError:
                break
            residual = math.fabs(x - y)
            x = y

            row = len(df)
            df.loc[row] = [y]
            if residual < TOR or x > 1e9:
                break
        return df

In [8]:
class Newton(NewtonMethod):

    def __init__(self):
        super(NewtonMethod, self).__init__()

    def f(self, x):
        sol = np.zeros(len(x))
        sol[0] = 3 * x[0] - math.cos(x[1] * x[2]) - 1.0 / 2.0
        sol[1] = pow(x[0], 2) - 81 * pow(x[1] + 0.1, 2) + math.sin(x[2]) + 1.06
        sol[2] = math.exp(-x[0] * x[1]) + 20 * x[2] + (10 * math.pi - 3.0) / 3.0
        return sol

    def jacobian(self, x):
        jac = np.zeros(shape=(3, 3))
        jac[0][0] = 3.0
        jac[0][1] = x[2] * math.sin(x[1] * x[2])
        jac[0][2] = x[1] * math.sin(x[1] * x[2])
        jac[1][0] = 2 * x[0]
        jac[1][1] = -162 * (x[1] + 0.1)
        jac[1][2] = math.cos(x[2])
        jac[2][0] = -x[1] * math.exp(-x[0] * x[1])
        jac[2][1] = -x[0] * math.exp(-x[0] * x[1])
        jac[2][2] = 20
        return jac

    def run(self, x):
        """
        given x_0 in R^3 as a starting point.

        :param x: x_0 as described
        :return: the minimizer x* of f
        """
        df = pd.DataFrame(columns=['x' + str(i + 1) for i in range(len(x))] + ['residual', 'actual-residual'])

        row = len(df)
        df.loc[row] = [xe for xe in x] + [np.nan, np.nan]

        while True:
            jac = self.jacobian(x)
            f = -self.f(x)
            y = linalg.solve(jac, f)
            nx = x + y
            residual = linalg.norm(x - nx, np.inf)
            x = nx

            row = len(df)
            df.loc[row] = [nxe for nxe in nx] + [residual, np.nan]
            if residual < TOR:
                break

        for i in range(len(df)):
            xk = np.array([df.loc[i][j] for j in range(len(x))])
            df.loc[i][4] = linalg.norm(x - xk, np.inf)

        return df

## Secant Method

In [9]:
class Secant1D(NewtonMethod):

    def __init__(self):
        super(NewtonMethod, self).__init__()

    def f(self, x):
        return math.cos(x) - x

    def jacobian(self, x):
        return

    def run(self, x0):
        df = pd.DataFrame(columns=['(ST) f(x)'])
        row = len(df)
        p0 = x0[0]
        p1 = x0[1]
        q0 = self.f(p0)
        q1 = self.f(p1)
        x = p0
        df.loc[row] = [x]
        for k in range(2, MAX_ITR):
            try:
                y = p1 - q1 * (p1 - p0) / (q1 - q0)
            except ValueError:
                break
            residual = math.fabs(y - p1)
            p0 = p1
            q0 = q1
            p1 = y
            q1 = self.f(y)

            row = len(df)
            df.loc[row] = [y]
            if residual < TOR or x > 1e9:
                break
        return df

## Quasi-Newton Method

In [10]:
class Broyden(NewtonMethod):

    def __init__(self):
        super(NewtonMethod, self).__init__()

    def f(self, x):
        sol = np.zeros(len(x))
        sol[0] = 3 * x[0] - math.cos(x[1] * x[2]) - 1.0 / 2.0
        sol[1] = pow(x[0], 2) - 81 * pow(x[1] + 0.1, 2) + math.sin(x[2]) + 1.06
        sol[2] = math.exp(-x[0] * x[1]) + 20 * x[2] + (10 * math.pi - 3.0) / 3.0
        return sol

    def jacobian(self, x):
        jac = np.zeros(shape=(3, 3))
        jac[0][0] = 3.0
        jac[0][1] = x[2] * math.sin(x[1] * x[2])
        jac[0][2] = x[1] * math.sin(x[1] * x[2])
        jac[1][0] = 2 * x[0]
        jac[1][1] = -162 * (x[1] + 0.1)
        jac[1][2] = math.cos(x[2])
        jac[2][0] = -x[1] * math.exp(-x[0] * x[1])
        jac[2][1] = -x[0] * math.exp(-x[0] * x[1])
        jac[2][2] = 20
        return jac

    def run(self, x):
        """
        given x_0 in R^3 as a starting point.

        :param x: x_0 as described
        :return: the minimizer x* of f
        """
        df = pd.DataFrame(columns=['x' + str(i + 1) for i in range(len(x))] + ['residual', 'actual-residual'])

        row = len(df)
        df.loc[row] = [xe for xe in x] + [np.nan, np.nan]
        A0 = self.jacobian(x)
        v = self.f(x)
        A = linalg.inv(A0)
        s = -A.dot(v)
        nx = x + s
        row = len(df)
        x = nx
        residual = linalg.norm(s, 2)
        df.loc[row] = [nxe for nxe in nx] + [residual, np.nan]

        for k in range(2, MAX_ITR):
            w = v
            v = self.f(x)
            y = v - w
            z = -A.dot(y)
            p = -s.transpose().dot(z)
            u = s.transpose().dot(A)
            A = A + 1 / p * np.outer((s + z), u)
            s = -A.dot(v)
            nx = x + s
            residual = linalg.norm(s, 2)
            x = nx

            row = len(df)
            df.loc[row] = [nxe for nxe in nx] + [residual, np.nan]
            if residual < TOR:
                break

        for i in range(len(df)):
            xk = np.array([df.loc[i][j] for j in range(len(x))])
            df.loc[i][4] = linalg.norm(x - xk, 2)

        return df

## Steepest Descent

In [11]:
class SteepestDescentMethod(object):

    def __init__(self):
        return

    @abstractmethod
    def f(self, x):
        return NotImplementedError('Implement f()!')

    @abstractmethod
    def g(self, x):
        return NotImplementedError('Implement g()!')

    @abstractmethod
    def grad_g(self, x):
        return NotImplementedError('Implement grad_g()!')

    @abstractmethod
    def jacobian(self, x):
        return NotImplementedError('Implement jacobian()!')

    @abstractmethod
    def run(self, x):
        return NotImplementedError('Implement run()!')

In [12]:
class SteepestDescent(SteepestDescentMethod):

    def __init__(self):
        super(SteepestDescentMethod, self).__init__()

    def f(self, x):
        sol = np.zeros(len(x))
        sol[0] = 3 * x[0] - math.cos(x[1] * x[2]) - 1.0 / 2.0
        sol[1] = pow(x[0], 2) - 81 * pow(x[1] + 0.1, 2) + math.sin(x[2]) + 1.06
        sol[2] = math.exp(-x[0] * x[1]) + 20 * x[2] + (10 * math.pi - 3.0) / 3.0
        return sol

    def g(self, x):
        sol = self.f(x)
        return sum([e * e for e in sol])

    def grad_g(self, x):
        return 2 * self.jacobian(x).transpose().dot(self.f(x))

    def jacobian(self, x):
        jac = np.zeros(shape=(3, 3))
        jac[0][0] = 3.0
        jac[0][1] = x[2] * math.sin(x[1] * x[2])
        jac[0][2] = x[1] * math.sin(x[1] * x[2])
        jac[1][0] = 2 * x[0]
        jac[1][1] = -162 * (x[1] + 0.1)
        jac[1][2] = math.cos(x[2])
        jac[2][0] = -x[1] * math.exp(-x[0] * x[1])
        jac[2][1] = -x[0] * math.exp(-x[0] * x[1])
        jac[2][2] = 20
        return jac

    def run(self, x):
        """
        given x_0 in R^3 as a starting point.

        :param x: x_0 as described
        :return: the minimizer x* of f
        """
        df = pd.DataFrame(columns=['x' + str(i + 1) for i in range(len(x))] + ['g', 'residual', 'actual-residual'])

        row = len(df)
        df.loc[row] = [xe for xe in x] + [self.g(x), np.nan, np.nan]

        while True:
            prev_x = x
            g1 = self.g(x)
            z = self.grad_g(x)
            z0 = linalg.norm(z, 2)
            if z0 == 0.0:
                print('Zero gradient')
                return x

            z /= z0
            alpha3 = 1
            g3 = self.g(x - alpha3 * z)
            while g3 >= g1:
                alpha3 /= 2.0
                g3 = self.g(x - alpha3 * z)
                if alpha3 < TOR / 2.0:
                    print('No likely improvement')
                    return x

            alpha2 = alpha3 / 2.0
            g2 = self.g(x - alpha2 * z)

            h1 = (g2 - g1) / alpha2
            h2 = (g3 - g2) / (alpha3 - alpha2)
            h3 = (h2 - h1) / alpha3

            alpha0 = (alpha2 - h1 / h3) / 2.0
            g0 = self.g(x - alpha0 * z)

            alpha = alpha0
            g = g0
            if g3 < g:
                alpha = alpha3
                g = g3

            x = x - alpha * z
            residual = linalg.norm(x - prev_x, np.inf)
            row = len(df)
            df.loc[row] = [nxe for nxe in x] + [g, residual, np.nan]
            if math.fabs(g - g1) < TOR:
                break

        for i in range(len(df)):
            xk = np.array([df.loc[i][j] for j in range(len(x))])
            df.loc[i][5] = linalg.norm(xk - x, np.inf)

        return df

# <font color='orange'> Run <font>

## 1-Dim

### Fixed Point

In [13]:
pd.options.display.float_format = '{:,.9f}'.format

In [14]:
x0 = np.array(1.5)
FixedPoint1D().run(x0).astype(np.float64)

,g1(x),g2(x),g3(x),g4(x),g5(x)
0,1.500000000,1.500000000,1.500000000,1.500000000,1.500000000
1,-0.875000000,0.816496581,1.286953768,1.348399725,1.373333333
2,6.732421875,2.996908806,1.402540804,1.367376372,1.365262015
3,-469.720012002,nan,1.345458374,1.364957015,1.365230014
4,"102,754,555.187385112",nan,1.375170253,1.365264748,1.365230013
5,"-1,084,933,870,531,746,352,594,944.000000000",nan,1.360094193,1.365225594,nan
6,"1,277,055,591,444,378,074,254,579,861,314,550,1...",nan,1.367846968,1.365230576,nan
7,nan,nan,1.363887004,1.365229942,nan
8,nan,nan,1.365916733,1.365230023,nan
9,nan,nan,1.364878217,1.365230012,nan


### Newton's method

In [15]:
pd.options.display.float_format = '{:,.10f}'.format

In [16]:
x0 = math.pi / 4.0

pd.concat([FixedPoint1D().run2(x0), Newton1D().run(x0)], axis=1)

,(FP) f(x),(NT) f(x)
0,0.7853981634,0.7853981634
1,0.7071067812,0.7395361335
2,0.7602445971,0.7390851781
3,0.7246674809,0.7390851332
4,0.7487198858,0.7390851332
5,0.7325608446,nan
6,0.7434642113,nan
7,0.7361282565,nan
8,0.7410736871,nan
9,0.7377441590,nan


### Secant Method

In [17]:
pd.options.display.float_format = '{:,.10f}'.format

In [18]:
x0 = 0.5
x1 = math.pi / 4.0
pd.concat([Newton1D().run(x1), Secant1D().run([x0, x1])], axis=1)

,(NT) f(x),(ST) f(x)
0,0.7853981634,0.5000000000
1,0.7395361335,0.7363841388
2,0.7390851781,0.7390581392
3,0.7390851332,0.7390851493
4,0.7390851332,0.7390851332
5,nan,0.7390851332


## Multi-Dim

### Fixed Point

In [19]:
pd.options.display.float_format = '{:,.8f}'.format

In [20]:
x0 = np.array([0.1, 0.1, -0.1])
FixedPoint().run(x0)

,x1,x2,x3,residual,actual-residual
0,0.10000000,0.10000000,-0.10000000,nan,0.42359878
1,0.49998333,0.00944115,-0.52310127,0.42310127,0.00944115
2,0.49999593,0.00002557,-0.52336331,0.00941558,0.00023546
3,0.50000000,0.00001234,-0.52359814,0.00023483,0.00001234
4,0.50000000,0.00000003,-0.52359847,0.00001230,0.00000031
5,0.50000000,0.00000002,-0.52359877,0.00000031,0.00000002
6,0.50000000,0.00000000,-0.52359878,0.00000002,0.00000000
7,0.50000000,0.00000000,-0.52359878,0.00000000,0.00000000


In [21]:
x0 = np.array([0.1, 0.1, -0.1])
FixedPointAcceleration().run(x0)

,x1,x2,x3,residual,actual-residual
0,0.10000000,0.10000000,-0.10000000,nan,0.42359878
1,0.49998333,0.02222979,-0.52304613,0.42304613,0.02222979
2,0.49997747,0.00002815,-0.52359807,0.02220164,0.00002815
3,0.50000000,0.00000004,-0.52359877,0.00002812,0.00000004
4,0.50000000,0.00000000,-0.52359878,0.00000004,0.00000000
5,0.50000000,0.00000000,-0.52359878,0.00000000,0.00000000


### Newton's method

In [22]:
pd.options.display.float_format = '{:,.10f}'.format

In [23]:
x0 = np.array([0.1, 0.1, -0.1])
Newton().run(x0)

,x1,x2,x3,residual,actual-residual
0,0.1000000000,0.1000000000,-0.1000000000,nan,0.4235987756
1,0.4998696729,0.0194668485,-0.5215204719,0.4215204719,0.0194668485
2,0.5000142402,0.0015885914,-0.5235569643,0.0178782572,0.0015885914
3,0.5000001135,0.0000124448,-0.5235984501,0.0015761466,0.0000124448
4,0.5000000000,0.0000000008,-0.5235987756,0.0000124440,0.0000000008
5,0.5000000000,0.0000000000,-0.5235987756,0.0000000008,0.0000000000


### Quasi-Newton method

In [24]:
pd.options.display.float_format = '{:,.8f}'.format

In [25]:
x0 = np.array([0.1, 0.1, -0.1])
Broyden().run(x0)

,x1,x2,x3,residual,actual-residual
0,0.10000000,0.10000000,-0.10000000,nan,0.59113105
1,0.49986967,0.01946685,-0.52152047,0.58656701,0.01957791
2,0.49998638,0.00873784,-0.52317457,0.01085640,0.00874814
3,0.50000660,0.00086727,-0.52357234,0.00788064,0.00086770
4,0.50000033,0.00003953,-0.52359769,0.00082816,0.00003954
5,0.50000000,0.00000019,-0.52359877,0.00003935,0.00000019
6,0.50000000,0.00000000,-0.52359878,0.00000019,0.00000000
7,0.50000000,0.00000000,-0.52359878,0.00000000,0.00000000


### Steepest Descent

In [26]:
pd.options.display.float_format = '{:,.6f}'.format

In [27]:
x0 = np.array([0, 0, 0])
SteepestDescent().run(x0)

,x1,x2,x3,g,residual,actual-residual
0,0.000000,0.000000,0.000000,111.974771,nan,0.523598
1,0.011218,0.010096,-0.522741,2.327617,0.522741,0.488749
2,0.137860,-0.205453,-0.522059,1.274058,0.215549,0.362107
3,0.266959,0.005511,-0.558494,1.068131,0.210964,0.233008
4,0.272734,-0.008118,-0.522006,0.468309,0.036488,0.227233
5,0.308689,-0.020403,-0.533112,0.381087,0.035956,0.191278
6,0.314308,-0.014705,-0.520923,0.318837,0.012188,0.185659
7,0.324267,-0.008525,-0.528431,0.287024,0.009958,0.175701
8,0.330809,-0.009678,-0.520662,0.261579,0.007768,0.169158
9,0.339809,-0.008592,-0.528080,0.238486,0.009000,0.160159
